(20%) Describe: Create a git repository and make it available online for the lecturer to clone. The repository should contain all your work for this assessment. Within the repository, create a jupyter [6] notebook that uses descriptive statistics and plots to describe the Boston House Prices [1] dataset. 

*Good summary of the dataset, repository well laid-out and organised. Reasonable commits to the repository.*

(20%) Infer: To the above jupyter notebook, add a section where you use inferential statistics to analyse whether there is a significant difference in median house prices between houses that are along the Charles river and those that aren’t. You should explain and discuss your findings within the notebook.

*Appropriate analysis of the relationship between the me- dian house price and whether the house borders the river, with good explanations of the analysis.*

(60%) Predict: Again using the same notebook, use keras [7] to create a neural network that can predict the median house price based on the other variables in the dataset. You are free to interpret this as you wish — for example, you may use all the other variables, or select a subset. 

*Well-trained neural network providing decent predictions of house prices based on the other variables. Good explanations of how to use the neural network and how it works.*

- Project plan in readme / github issues
- Explanation of project in readme


[1] Boston Standard Metropolitan Statistical Area. Boston house prices dataset. https://www.kaggle.com/c/boston-housing.

[2] SciPy developers. The scipy python package. https://www.scipy.org/.

[3] Python Software Foundation. Welcome to python.org. https://www.python.org/.

[4] GMIT. Quality assurance framework. https://www.gmit.ie/general/quality-assurance-framework.

[5] GitHub Guides. Mastering markdown. https://guides.github.com/features/mastering-markdown/.

[6] Project Jupyter. Project jupyter. https://jupyter.org/.

[7] Keras Team. Keras: The python deep learning library. https://keras.io/.